In [2]:
using DataStructures
using FileIO
using Meshes
using MeshIO
using Makie
using GLMakie
using GeometryBasics
using ProgressBars

In [3]:
include("contractions.jl")
include("visualisation.jl")

visualize (generic function with 2 methods)

In [ ]:

function CreateSimplicialComplex2D(mesh)
    coords = Vector{Vector{Float32}}()
    anticoords = Dict{Vector{Float32},Int}()

    triangles_ = Vector{Tuple{Int,Int,Int}}()

    for triangle in mesh
        for vertex in triangle
            if !haskey(anticoords, vertex)
                push!(coords, collect(vertex))
                anticoords[collect(vertex)] = length(coords)
            end
        end
        push!(triangles_, (anticoords[collect(triangle[1])], anticoords[collect(triangle[2])], anticoords[collect(triangle[3])]))
    end

    SimplicialComplex2D(triangles_, coords)
end

function loadsc(path)
    data = load(path)
    data_triangles = Array([Array([Tuple([round(z, digits=2) for z in y]) for y in x]) for x in data])
    CreateSimplicialComplex2D(data_triangles)
end


loadsc (generic function with 1 method)

In [10]:
bunidata = loadsc("bunny/reconstruction/bun_zipper.ply")
# fuzz = 1
# for i in 1:length(bunidata.coords)
#     bunidata.coords[i][1] *= rand() * fuzz
#     bunidata.coords[i][2] *= rand() * fuzz
#     bunidata.coords[i][3] *= rand() * fuzz
# end

SimplicialComplex2D(Vector{Float32}[[-0.09231, 0.13236, 0.01822], [-0.09218, 0.13235, 0.01722], [-0.09277, 0.13099, 0.01723], [-0.08717, 0.11039, 0.01836], [-0.08896, 0.11186, 0.01834], [-0.08744, 0.11045, 0.01734], [-0.08784, 0.10502, 0.01437], [-0.08724, 0.10636, 0.01436], [-0.08725, 0.10635, 0.01336], [-0.08785, 0.10502, 0.01337]  …  [-0.06682, 0.03604, 0.03391], [-0.06671, 0.03757, 0.03519], [-0.06666, 0.03607, 0.03539], [-0.06658, 0.03628, 0.03652], [-0.06666, 0.03607, 0.03539], [-0.06671, 0.03757, 0.03519], [-0.06682, 0.03604, 0.03391], [-0.06682, 0.03604, 0.03391], [-0.06671, 0.03757, 0.03519], [-0.06673, 0.03741, 0.03372]], Set([185983, 92533, 137206, 76914, 150095, 37100, 173643, 90240, 3406, 185066  …  53638, 89207, 98986, 99296, 192841, 202471, 98971, 150325, 161331, 175679]), Dict{Int64, Set{Tuple{Int64, Int64}}}(185983 => Set([(185983, 185984), (185983, 185985)]), 92533 => Set([(92533, 92535), (92533, 92534)]), 137206 => Set([(137206, 137207), (137206, 137208)]), 76914 => 

In [11]:
buni = initialContractedSimplicialComplex2D(bunidata)

ContractedSimplicialComplex2D(SimplicialComplex2D(Vector{Float32}[[-0.09231, 0.13236, 0.01822], [-0.09218, 0.13235, 0.01722], [-0.09277, 0.13099, 0.01723], [-0.08717, 0.11039, 0.01836], [-0.08896, 0.11186, 0.01834], [-0.08744, 0.11045, 0.01734], [-0.08784, 0.10502, 0.01437], [-0.08724, 0.10636, 0.01436], [-0.08725, 0.10635, 0.01336], [-0.08785, 0.10502, 0.01337]  …  [-0.06682, 0.03604, 0.03391], [-0.06671, 0.03757, 0.03519], [-0.06666, 0.03607, 0.03539], [-0.06658, 0.03628, 0.03652], [-0.06666, 0.03607, 0.03539], [-0.06671, 0.03757, 0.03519], [-0.06682, 0.03604, 0.03391], [-0.06682, 0.03604, 0.03391], [-0.06671, 0.03757, 0.03519], [-0.06673, 0.03741, 0.03372]], Set([185983, 92533, 137206, 76914, 150095, 37100, 173643, 90240, 3406, 185066  …  53638, 89207, 98986, 99296, 192841, 202471, 98971, 150325, 161331, 175679]), Dict{Int64, Set{Tuple{Int64, Int64}}}(185983 => Set([(185983, 185984), (185983, 185985)]), 92533 => Set([(92533, 92535), (92533, 92534)]), 137206 => Set([(137206, 137207),

In [13]:
visualize(buni)

GLMakie.Screen(...)

In [ ]:
include("contractions.jl")

issafe (generic function with 1 method)

In [ ]:
error(buni, (1, 2))

0.017894885f0

In [7]:
pq = PriorityQueue()
for (edge, triangle) in ProgressBar(buni.contracted._edge_to_triangles)
    pq[edge] = error(buni, edge)
end

0.0%┣                                   ┫ 0/208.4k [00:14<-813:-51:-37, -14s/it]
0.0%┣                                       ┫ 1/208.4k [00:25<Inf:Inf, InfGs/it]
0.0%┣                                       ┫ 11/208.4k [00:25<145:35:05, 3s/it]
1.8%┣▊                                      ┫ 3.7k/208.4k [00:25<22:59, 148it/s]
3.5%┣█▍                                     ┫ 7.3k/208.4k [00:25<11:37, 289it/s]
5.2%┣██                                    ┫ 10.9k/208.4k [00:25<07:38, 432it/s]
7.1%┣██▊                                   ┫ 14.7k/208.4k [00:25<05:34, 580it/s]
8.8%┣███▍                                  ┫ 18.4k/208.4k [00:25<04:23, 723it/s]
10.5%┣███▉                                 ┫ 21.8k/208.4k [00:25<03:38, 855it/s]
12.5%┣████▌                               ┫ 26.0k/208.4k [00:26<02:59, 1.0kit/s]
14.3%┣█████▏                              ┫ 29.8k/208.4k [00:26<02:33, 1.2kit/s]
16.0%┣█████▊                              ┫ 33.4k/208.4k [00:26<02:14, 1.3kit/s]
17.8%┣██████▍               

In [8]:
for i in ProgressBar(1:3)
    first = dequeue!(pq)
    while !issafe(buni.contracted, first)
        first = dequeue!(pq)
    end

    new_vertex = contract!(buni, first)
    for edge in buni.contracted._vertex_to_edges[new_vertex]
        pq[edge] = error(buni, edge)
        for triangle in buni.contracted._edge_to_triangles[edge]
            pq[triangle] = error(buni, triangle)
        end
    end
end

0.0%┣                                                ┫ 0/3 [00:00<00:00, -0s/it]


BoundsError: BoundsError: attempt to access Set{Int64} with 1 element at index [2]